<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/DLwithPytorch/ch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망 더 깊이 알아보기
  - 신경망의 다양한 구성요소
  - 비선형 활성화(activations)
  - 심층 학습을 이용한 이미지 분류
<br/>

#### 딥러닝 알고리즘 훈련단계(review)
> 1. 데이터 파이프라인 구축하기
> 2. 네트워크 아키텍처 구축하기
> 3. 손실 함수를 이용하여 아키텍처 평가
> 4. 최적화 알고리즘을 이용하여 가중치들 최적화

- 단순한 선형 모델로 구성된 네트워크는 컴퓨터 비전 및 자연어 처리와 같은 여러 분야의 복잡한 문제를 해결하는데 필요한 아키텍처를 구축할 때는 매우 복잡
- 파이토치, 텐서플로우 같은 딥러닝 프레임위크는 이러한 복잡성을 추상화하는 더 높은 수준의 기능을 제공
  - 레이어(layer):    
    - 데이터를 입력받고 변환을 적용하여 데이터를 출력
    - 각 layer는 고유한 parameter(가중치)를 가진다.

---
#### Layers : Fundamental blocks of Neural Network
- 선형 레이어: y = w * x + b




In [7]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torchvision import models
from torchvision.datasets import ImageFolder

In [11]:
# linear_layer 함수는 크기 5의 tensor를 받아서 선형변환을 한 후 크기 3의 tensor 출력
linear_layer = Linear(in_features=5,out_features=3,bias=True)

inp = Variable(torch.randn(1,5))
linear_layer(inp)

tensor([[0.8051, 0.7230, 0.4042]], grad_fn=<AddmmBackward>)

In [12]:
# .weight, bias로 레이어의 훈련 가능한 poarameter에 접근 가능
linear_layer.weight

Parameter containing:
tensor([[-0.3886,  0.3273,  0.0957,  0.4225, -0.1291],
        [ 0.0211,  0.1651,  0.0130, -0.2560,  0.2588],
        [-0.3429,  0.1056, -0.3616,  0.2794, -0.2087]], requires_grad=True)

In [13]:
linear_layer.bias

Parameter containing:
tensor([-0.0997,  0.3207,  0.3656], requires_grad=True)

In [ ]:
# layer의 output을 다른 layer로 전달
linear_layer = Linear(5,3)
linear_layer_2 = Linear(3,2)
linear_layer_2(linear_layer(inp))

---
## 비선형 활성화
 - 단순히 다중 (선형)레이어를 쌓는 것 만으로는 알고리즘이 어떤 새로운 것을 학습하는데 도움이 되지 않는다.
 - 비선형 함수: 수학적 변환 ex) igmoid, Tanh, ReLU, Leaky ReLU
>   #### - sigmoid 함수   
>   >  실수 값을 취해서 0과 1사이의 값 출력   
>   >  gradient vanishing 문제
>   > <img src="https://t1.daumcdn.net/cfile/tistory/275BAD4F577B669920" width="400px" height="250px" alt="sigmoid"></img><br/>
>   #### - Tanh 함수   
>  >   -1과 1 사이의 실수 값 제곱   
>  >   gradient exploding 문제
>  >   <img src="https://mlnotebook.github.io/img/transferFunctions/tanh.png" width="400px" height="250px" alt="tanh function"></img><br/>
>   #### - ReLU   
>  >   f(x) = max(0,x)   
>  >   음수는 0, 양수는 그대로
>  >   <img src="https://learnopencv.com/wp-content/uploads/2017/10/relu-activation-function-1.png" width="400px" height="250px" alt="tanh function"></img><br/>
>  >   - gradient descent의 속도가 빨라 올바른 W 빠르게 찾는다.
>  >   - 단순히 임계값만 정해 계산 비용이 저렴(sigmoid, tanh도)
>  >   - 역전파(backward propagation)을 수행하는 동안 큰 기울기가 전달되면 반응이 없는 경우 종종 발생 -> learning rate 선택하여 제어

- learning rate: parameter에 대한 변경 비율
- pytorch에서 모든 네트워크는 nn.Moduel을 서브클래싱하는 클래스들로 구현, init, foward를 구현해야 함
  - init: 모든 레이어 초기화(생성자 역할)
  - forward: init애서 초기화한 레이어들로 입력 데이터를 전달하고 최종 출력을 반환



In [ ]:
class MyFirstNetwork(nn.Module):    # 부모클래스의 이름을 인수로 전달하여 서브클래스 생성
    def __init__(self, input_size, hidden_size, output_size):
        super(MyFirstNetwork,self).__init__() # super: 자식클래스의 인수를 부모클래스(nn.Module)로 전달
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def __forward__(self,input): 
        out = self.layer1(input) 
        out = nn.ReLU(out)
        out = self.layer2(out) 
        return out

my_network = MyFirstNetwork(input_size = 3, hidden_size = 2, output_size = 1)       

---
#### loss 함수
gradient descent는 일반적으로 스칼라 값을 받아들이기 때문에, loss 함수는 스칼라 값을 생성해야 함 -> 여러 loss를 하나의 스칼라 값으로 합침
>   #### - MSE(Mean Square Error): 평균 제곱 오차
>   - 회귀 문제를 위한 손실함수

```python
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()
```
>   #### - cross-entropy: 교차 엔트로피
>   - 다중 클래스 분류 문제를 위해 사용되는 손실 함수
>   - softmax layer와 같이, 더하면 1이 되는 확률 값을 예측하는 분류 네트워크의 loss 계산
>   - 예측된 확률이 실제 확률을 벗어날 때 loss 증가
 
 ```python
 def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)
```
```python
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
```

#### Optimizer(최적화기)
  - ex) ASGD, Adadelta, Adagrad, Adam, Adamax, LBFGS, RMSprop, SGD, SparseAdam   
  - SGD optimizer(학습 가능한 모든 parameters, learning rate)



In [ ]:
optimizer = torch.optim.SGD(my_network.parameters(), lr = 0.01)
for input, target in torch.utils.data.Dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

## 딥러닝을 이용한 이미지 분류


In [3]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import warnings
warnings.filterwarnings("ignore")
import time
%matplotlib inline

- 이미지 데이터 다운로드

In [4]:
def imshow(inp):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
path= 'content/gdrive/MyDrive/Dog-Cat-Classifier'

In [11]:
cat_files = [f for f in glob.glob('/content/gdrive/MyDrive/Dog-Cat-Classifier/Data/Train_Data/cat/*.jpg')]
dog_files = [f for f in glob.glob('/content/gdrive/MyDrive/Dog-Cat-Classifier/Data/Train_Data/dog/*.jpg')]
files = dog_files + cat_files
print(f'Total no of images {len(files)}')

Total no of images 1399


- test dataset(검증데이터셋)을 생성하는데 사용하는 셔플된 인덱스 생성

- train imgs, test imgs를 저장할 test directory 생성


<br/>

---
* 코드: <https://github.com/PacktPublishing/Deep-Learning-with-PyTorch-1.x/blob/master/Chapter03/Chapter03a.ipynb>